# Prep

In [ ]:
#import tensorflow_datasets as tfds
# import tensorflow as tf
import random 

import time
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from data_preparation import get_all_classes, create_silence, get_validation_list, get_test_val_lists, make_spec, create_sets

In [ ]:
train_dir = "C://Users//jakub//Desktop//PD//sem10//deep//tensorflow-speech-recognition-challenge//train"

In [ ]:
background_noise_dir = os.path.join(train_dir, "audio", "_background_noise_")

if os.path.exists(background_noise_dir):
    shutil.move(background_noise_dir, train_dir)

In [ ]:
create_silence(train_dir)
classes = get_all_classes(train_dir)
print(classes)

In [ ]:
validation_list = get_validation_list(train_dir)
training_list = get_test_val_lists(train_dir,validation_list)

In [ ]:
X_train, y_train = create_sets(training_list, classes,train_dir)

In [ ]:
X_val, y_val = create_sets(validation_list, classes,train_dir)

# Tests

## Model tests

In [ ]:
# TODO

## Approaches tests

In [ ]:
# TODO

## Augmentation tests

In [ ]:
# TODO

## Hyperparametres tests

In [ ]:
# TODO